### STATE_DATA_Run

Prepares BATCH code to run STATE_DATA project data and analysis.

Batch code using BSUB is submitted through BASH terminal


New cell test

In [ ]:
#Unzip update code from Github ZIP file/

In [ ]:
%%bash
unzip cpq_tools-main.zip 'cpq_tools-main/*' -d ~/ && mv ~/cpq_tools-main/* ~/ && rmdir ~/cpq_tools-main

In [4]:
#Import code generation functions
from cpq_tools import write_bsub

In [5]:
def _project_bsub(project, verbose = False):
    """Generate BATCH job scheduler .sh files and bsub call for running each part of the project"""
    #Define dictionary of project files
    project_files = {'processing':'STATE_DATA_process.py',
                'analysis':'STATE_DATA_analysis.py'}
    code_file = 'out_dir/' + project_files[project]
    bsub_data = write_bsub(project, code_file)
    if verbose:
        print(bsub_data['bsub_code'])
    return(bsub_data['out_command'])
    

In [6]:
_project_bsub('processing', verbose=True)

#!/bin/bash
#BSUB -J processing # LSF Job Name
#BSUB -o job_output.%J # Name of the job output file
#BSUB -e job_error.%J # Name of the job error file
module unload python
module load python/3.8
python --version
python $HOME/out_dir/STATE_DATA_process.py



'BSUB < processing.sh'

In [4]:
_project_bsub('analysis')

'BSUB < analysis.sh'